In [1]:
import pandas as pd 
from sklearn.preprocessing import LabelEncoder
import numpy as np


ModuleNotFoundError: No module named 'sklearn'

In [ ]:
df = pd.read_csv("../diabetes+130-us+hospitals+for+years+1999-2008/diabetic_data.csv")

## data_cleaning

In [ ]:
df.head()

In [ ]:
#there are other typesof Null values in the data but starting with checking 
#the ones system recognize 

df.isnull().sum()

In [ ]:
print(f"max_glu_serum column is {(df['max_glu_serum'].isnull().sum()*100 / len(df['max_glu_serum'])).round(2)}% empty")
print(f"A1Cresult column is {(df['A1Cresult'].isnull().sum()*100 / len(df['A1Cresult'])).round(2)}% empty")

#as these columns are mostly empty, dropping them & creating a copy of original df 
df_cleaned = df.copy()
df_cleaned.drop(columns=["max_glu_serum","A1Cresult"],inplace=True)

In [ ]:
#assigning raw data columns per dtype 

numeric_columns = df_cleaned.select_dtypes("number")
categorical_columns = df_cleaned.select_dtypes("object")

In [ ]:
def replace_question_marks_with_nan(df):
    """ Defining a function to replace '?' with NaN """
    return df.replace('?', np.NaN, inplace=True)


replace_question_marks_with_nan(df_cleaned)

In [ ]:
df_cleaned.shape

In [ ]:
#checking for duplicated but result is 0 
df_cleaned.duplicated().sum()

In [ ]:
# columns 'diag_1' 'diag_2', 'diag_3' have a lot unique columns which are
#codes for spesific diagnoses. Will drop these columns but later 
#might look into again by themselves. 

#also medical_specialty & payer_code columns are irrelevant, so will drop them as well 

#and to keep things simple, dropping discharge_disposition_id & admission_source_id 

columns_to_remove = ['diag_1',
                     'diag_2',
                     'diag_3',
                     'medical_specialty',
                     'payer_code',
                     'discharge_disposition_id',
                     'admission_source_id',
                     'admission_type_id'
                    ]


df_cleaned.drop(columns_to_remove,
                axis=1,
                inplace=True)

In [ ]:
df_cleaned.isnull().sum()

In [ ]:
df_cleaned.to_csv('../partially_cleaned.csv', index=False)

In [ ]:
df['readmitted'].value_counts(normalize=True)

In [ ]:
a = pd.crosstab(df['readmitted'], df['age']).apply(lambda c: c / c.sum() * 100, axis=0).round(2)


In [ ]:
a

In [ ]:
def compare_columns_to_readmitted(df):
    for column in df.columns:
        if column != 'readmitted':
            cross_tab = pd.crosstab(df['readmitted'], df[column]).apply(lambda c: c / c.sum() * 100, axis=0).round(2)
            print(f"Cross-tabulation of 'readmitted' and '{column}':")
            print(cross_tab)
            print("\n")

In [ ]:
compare_columns_to_readmitted(df.select_dtypes('object'))

In [ ]:
for column in binary_columns:
    print(column,df_cleaned[column].unique())

In [ ]:
def replace_binary_values(df, columns):
    """defining a function to change binary columns into numeric 0 -1 """
    for column in binary_columns:
        df[column] = df[column].replace({'No': 0,
                                         'Steady': 1,
                                         'Yes': 1,
                                         'Ch':1})


replace_binary_values(df_cleaned,binary_columns)

In [ ]:
#now will try to assing numeric values for other columns that has 
#more than 2 unique (yes/no) values 

for column in other_columns:
    print(column,df_cleaned[column].unique())

In [ ]:
#also removing these column names from my list 

other_columns = [col for col in other_columns if col not in columns_to_remove]

In [ ]:
for column in other_columns: 
    print(column,df_cleaned[column].unique())

In [ ]:
#checking for binary columns and non-binary columns to check for null values 
#that system does not check fot categorical columns 

binary_columns = []
other_columns = []
for column in categorical_columns.columns:
    if len(categorical_columns[column].unique()) == 2: 
        binary_columns.append(column)
    else: 
        other_columns.append(column)

#using label encoding for age column and dropping the original one 

#label_encoder = LabelEncoder()

#df_cleaned['age_encoded'] = label_encoder.fit_transform(df_cleaned['age'])

for label, original_bin in enumerate(label_encoder.classes_):
    print(f"{original_bin}: {label}")

df_cleaned.drop('age', axis=1, inplace=True)

# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Fit label encoder and transform 'weight' column
df['weight_encoded'] = label_encoder.fit_transform(df['weight'])

# Print the mapping between original bins and encoded values
for label, original_bin in enumerate(label_encoder.classes_):
    print(f"{original_bin}: {label}")

# Optionally, you can drop the original 'weight' column
# df.drop('weight', axis=1, inplace=True)

In [ ]:
len(df_cleaned[df_cleaned['weight'] == '?'])/len(df_cleaned[df_cleaned['weight'] != '?'])